In [1]:
import numpy as np
from tensorflow import keras
import pickle
import random
from pathlib import Path
from vectorization import nw_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
import tensorflow_text as tf_text
import keras_nlp
import pickle
from pathlib import Path

In [2]:
num_samples = 142000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "/lexique/lexique.csv"

def construct_tuples(path):
  input_texts = []
  target_texts = []
  with open(data_path, "r", encoding="utf-8") as f:
      lines = f.readlines()
  for line in lines[: min(num_samples, len(lines) - 1)]:
      input_text, target_text = line.split(",")
      input_texts.append(input_text)
      target_texts.append(target_text)

  tuples = []

  for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    tuples.append([input_text, target_text, 0])

  return tuples

def find_best_match(word, tuples):
  tuples = tuples.copy()

  for idx_char, char in enumerate(reversed(word)):
    for i, line in enumerate(tuples):
      if len(line[1]) >= idx_char + 1:
        if char == line[1][-1-idx_char]:
          line[2] += 1/(idx_char)    

  return sorted(tuples,key=lambda x:x[2],reverse=True)



In [9]:
def find_best_match2(word, tuples):
    tuples = tuples.copy()
    
    for line in tuples:
        line[2] = 0
        packed = zip(word[::-1], line[1][::-1])
            
        for i, pair in enumerate(packed):
            if pair[0] == pair[1]:
                line[2] += 1 / (i + 1)
            else:
                break
        
    return sorted(tuples,key=lambda x:x[2],reverse=True)

In [4]:
APP_DIR = Path.home() / ".lafontaine"
VECTORIZER = APP_DIR / "vectorizer.pkl"

with VECTORIZER.open('rb') as stream:
    from_disk = pickle.load(stream)

vectorizer = TextVectorization.from_config(from_disk['config'])
vectorizer.set_weights(from_disk['weights'])

seq_length = from_disk['seq_length']
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
index_lookup = dict(zip(range(len(vocab)), vocab))

In [5]:
# Restore the model and construct the encoder and decoder.
phonetic_model = keras.models.load_model("s2s")

# Restore sentence model
sentence_model = keras.models.load_model("sentence_model", compile=False)

/usr/local/lib/python3.8/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [6]:
batch_size = 128  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 142000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "/lexique/lexique.csv"

In [13]:
def sample_token(logits):
    logits, indices = tf.math.top_k(logits, k=5, sorted=True)
    indices = np.asarray(indices).astype("int32")
    preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
    preds = np.asarray(preds).astype("float32")
    return np.random.choice(indices, p=preds)

def generate_text(prompt, response_length=20):
    decoded_sample = prompt
    for i in range(response_length-1):
        tokenized_prompt = vectorizer([decoded_sample])[:, :-1]
        predictions = sentence_model.predict([tokenized_prompt], verbose=0)
        sample_index = len(decoded_sample.strip().split())-1
        sampled_token = sample_token(predictions[0][sample_index])
        sampled_token = index_lookup[sampled_token]
        decoded_sample += " " + sampled_token
    return decoded_sample

tuples = construct_tuples(data_path)

def encode_input_text(input_text):
  encoder_input_data = np.zeros(
      (1, max_encoder_seq_length, num_encoder_tokens), dtype="float32"
  )

  for t, char in enumerate(input_text):
      encoder_input_data[0, t, input_token_index[char]] = 1.0
  encoder_input_data[0, t + 1 :, input_token_index[" "]] = 1.0
  return encoder_input_data

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index[","]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split(",")
    # We use "," as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "," + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    #decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    #decoder_target_data[i, t:, target_token_index[" "]] = 1.0

    # Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2


decoder_inputs_t = model.input[1]  # input_2
decoder_inputs = tf.identity(decoder_inputs_t)

decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

def make_poem(input: str, length: int):
    input_seq = encode_input_text(input)
    phonetic = decode_sequence(input_seq)
    print("Phonetic:", phonetic)
    sorted_tuples = find_best_match2(phonetic, tuples)
    print(sorted_tuples[0:5])
    rime = random.choice(sorted_tuples[0:10])
    print("Rime found: " + rime[1] + " -> " + rime[0])
    sentence1 = generate_text(input, length)     
    sentence2 = generate_text(rime[0], length)
    
    print("Poem:")
    print(' '.join(sentence1.split(' ')[::-1]))
    print(' '.join(sentence2.split(' ')[::-1]))


Number of samples: 142000
Number of unique input tokens: 53
Number of unique output tokens: 42
Max sequence length for inputs: 25
Max sequence length for outputs: 23


In [ ]:
make_poem("canabisse", 10)

1/1 [==============================] - 0s 17ms/step
Phonetic: kanabis

[['cannabis', 'kanabis\n', 2.7178571428571425], ['abysse', 'abis\n', 2.283333333333333], ['abysses', 'abis\n', 2.283333333333333], ['bis', 'bis\n', 2.083333333333333], ['bis', 'bis\n', 2.083333333333333]]
Rime found: ibis
 -> ibis
